In [39]:
import pandas as pd
import numpy as np
import re
from typing import List, Tuple, Dict, Any, Set, Callable, Hashable
import matplotlib.pyplot as plt
import plotly.graph_objects as go

In [2]:
column_names = [
  # 'rec_id',
  'given_name',
  'surname',
  'street_number',
  'address_1',
  'address_2',
  'suburb',
  # 'postcode',
  'state',
  'date_of_birth',
  # 'age',
  'phone_number',
  # 'soc_sec_id',
  # 'blocking_number'
]

In [3]:

def clean_address(address: str) -> str:
    # Список слов и сокращений для удаления
    words_to_remove = [
        "улица", "ул\\. ?", "проспект", "пр-т\\. ?", "пр\\. ?", "бульвар", "б-р\\. ?", "переулок", "пер\\. ?", "набережная", "наб\\. ?",
        "шоссе", "площадь", "пл\\. ?", "дом", "д\\. ?", "квартира", "кв\\. ?", "корпус", "корп\\. ?", "строение", "стр\\. ?", "область",
        "обл\\. ?", "город", "г\\. ?", "поселок", "пос\\. ?", "деревня", "дер\\. ?",
        "street", "st\\. ?", "avenue", "ave\\. ?", "boulevard", "blvd\\. ?", "alley", "al\\. ?", "drive", "dr\\. ?",
        "square", "sq\\. ?", "house", "h\\. ?", "apartment", "apt\\. ?", "building", "bldg\\. ?", "county", "co\\. ?",
        "city", "ct\\. ?", "village", "vil\\. ?", "township", "twp\\. ?", "road", "rd\\. ?"
    ]

    # Создаем регулярное выражение из списка слов и сокращений
    pattern = r'\b(?:{})\b'.format('|'.join(words_to_remove))

    # Заменяем найденные слова и сокращения на пустую строку
    cleaned_address = re.sub(pattern, '', address, flags=re.IGNORECASE)

    # Удаляем лишние пробелы и возвращаем очищенную строку
    return re.sub(r'\s+', ' ', cleaned_address).strip()

In [4]:
def get_key(x: str) -> str:
    # Формируем ключ из значений полей
    return  '-'.join([x.split('-')[0], x.split('-')[1]])

In [5]:
data = pd.read_csv('test.csv', dtype=str)
data = data.applymap(lambda x:  x.strip() if isinstance(x, str) else x )
# data['address_1'] = data['address_1'].apply(lambda x:  clean_address(x) if isinstance(x, str) else x )
# data['address_2'] = data['address_2'].apply(lambda x:  clean_address(x) if isinstance(x, str) else x )
# data = data.applymap(lambda x:  clean_address(x) if isinstance(x, str) else x )
data = data.replace({np.nan:None})
data['rec_common_id'] = data.apply(lambda x: get_key(x['rec_id']), axis=1)
data_list = data.to_dict('records')

In [6]:
data_by_ids: Dict[str, List[Dict[Hashable, Any]]] = {}
expected_res: Dict[str, Set[str]] = {}
origs: Dict[str, Dict[Hashable, Any]] = {}
for record in data_list:
    if record['rec_common_id'] not in data_by_ids:
        data_by_ids[record['rec_common_id']] = []
        expected_res[record['rec_common_id']] = set()
    data_by_ids[record['rec_common_id']].append(record)
    expected_res[record['rec_common_id']].add(record['rec_id'])
    if 'org' in record['rec_id']:
        origs[record['rec_id']] = record

In [7]:
from distances.levenshtein import levenstein_similarity, levenshtein_distance_memopt
from distances.jaro import jaro_winkler_similarity
from distances.damerau_levenstein import damerau_levenshtein_similarity, damerau_levenshtein_distance_memopt
from distances.jaccard import jaccard_similarity_str

In [8]:
def get_sim_mean(a: Dict[Hashable, Any], b: Dict[Hashable, Any], sim: Callable[[str, str], float]) -> float:
  sm = 0
  cnt = 0
  for k in column_names:
    if a[k] is not None and b[k] is not None and isinstance(a[k], str) and a[k] != '' and isinstance(b[k], str) and b[k] != '':
      sm += sim(a[k], b[k])
      cnt += 1
  if cnt == 0:
    return 0
  return sm / cnt
def get_lev_sim(a: Dict[Hashable, Any], b: Dict[Hashable, Any]) -> float:
  return get_sim_mean(a, b, levenstein_similarity)
def get_dam_lev_sim(a: Dict[Hashable, Any], b: Dict[Hashable, Any]) -> float:
  return get_sim_mean(a, b, damerau_levenshtein_similarity)
def get_jaro_sim(a: Dict[Hashable, Any], b: Dict[Hashable, Any]) -> float:
  return get_sim_mean(a, b, jaro_winkler_similarity)
def get_jaccard_sim(a: Dict[Hashable, Any], b: Dict[Hashable, Any]) -> float:
  return get_sim_mean(a, b, jaccard_similarity_str)

In [9]:
def test_distances(data: List[Dict[Hashable, Any]], expected_res: Dict[str, Set[str]], similarity: Callable[[Dict[Hashable, Any], Dict[Hashable, Any]], float],threshold: float = 0.85) -> Tuple[int, int, int]:
  tp = 0
  fp = 0
  fn = 0
  res = []
  for i in range(len(data)):
    for j in range(i + 1, len(data)):
      sim = similarity(data[i], data[j])
      if sim > threshold:
        res.append((data[i]['rec_id'], data[j]['rec_id'], sim))
  res_by_ids = {}
  for (a_id, b_id, sim) in res:
    a_key = get_key(a_id)
    b_key = get_key(b_id)
    if a_key != b_key:
      print(f'ERROR: {a_id}, {b_id}, {sim}')
      fp += 1
      continue
    if a_key not in res_by_ids:
      res_by_ids[a_key] = set()
    res_by_ids[a_key].add(a_id)
    res_by_ids[a_key].add(b_id)
  for key in expected_res:
    if key not in res_by_ids:
      if len(expected_res[key]) > 1:
        # print(f'ERROR: not found {key}')
        fn += len(expected_res[key])
      continue
    # if len(expected_res[key]) != len(res_by_ids[key]):
    #   print(f'ERROR: not full {key}')
    fn += len(expected_res[key]) - len(res_by_ids[key])
    tp += len(res_by_ids[key])
  return tp, fp, fn

In [10]:
# (tp, fp, fn) = test_distances(data_list, expected_res, get_lev_sim)
# print('LEVENSTEIN')
# print(f'tp: {tp}, fp: {fp}, fn: {fn}')
# precision = tp / (tp + fp)
# recall = tp / (tp + fn)
# print(f'precision: {precision}')
# print(f'recall: {recall}')
# print(f'f1: {2 * precision * recall / (precision + recall)}')

In [11]:
# (tp, fp, fn) = test_distances(data_list, expected_res, get_jaro_sim)
# print('JARO')
# print(f'tp: {tp}, fp: {fp}, fn: {fn}')
# precision = tp / (tp + fp)
# recall = tp / (tp + fn)
# print(f'precision: {precision}')
# print(f'recall: {recall}')
# print(f'f1: {2 * precision * recall / (precision + recall)}')

In [12]:
# (tp, fp, fn) = test_distances(data_list, expected_res, get_dam_lev_sim)
# print('DAMERAU LEVENSTEIN')
# print(f'tp: {tp}, fp: {fp}, fn: {fn}')
# precision = tp / (tp + fp)
# recall = tp / (tp + fn)
# print(f'precision: {precision}')
# print(f'recall: {recall}')
# print(f'f1: {2 * precision * recall / (precision + recall)}')

In [13]:
# (tp, fp, fn) = test_distances(data_list, expected_res, get_jaccard_sim)
# print('JACCARD')
# print(f'tp: {tp}, fp: {fp}, fn: {fn}')
# precision = tp / (tp + fp)
# recall = tp / (tp + fn)
# print(f'precision: {precision}')
# print(f'recall: {recall}')
# print(f'f1: {2 * precision * recall / (precision + recall)}')

In [14]:
from dcs import dcs_plus_plus
from dm_soundex import encode

In [15]:
def dcs_key(r: Dict[Hashable, Any]) -> str:
  name = r['given_name']
  sur = r['surname']
  if name is None:
    name = ''
  # elif name != '':
  #   name = encode(name, max_length=10, zero_pad=True)
  if sur is None:
    sur = ''
  # elif sur != '':
  #   sur = encode(sur, max_length=10, zero_pad=True)
  name.replace(' ', '')
  sur.replace(' ', '')
  s = f'{name}{sur}'
  if s == '':
    return '0'*10
  # print(encode(r['surname'], max_length=10, zero_pad=True))
  return s
def is_duplicate(r1: Dict[Hashable, Any], r2: Dict[Hashable, Any]) -> bool:
  return get_jaccard_sim(r1, r2) > 0.7

In [41]:
def get_stats(res: List[Tuple[Dict[Hashable, Any], Dict[Hashable, Any]]], expected_res) -> Tuple[int, int, int, float, float, float]:
  tp = 0
  fp = 0
  fn = 0
  res_by_ids = {}
  for (a, b) in res:
    a_id = a['rec_id']
    b_id = b['rec_id']
    if a_id == b_id:
      print(f'ERROR: {a_id}, {b_id}')
      continue
    a_key = get_key(a_id)
    b_key = get_key(b_id)
    if a_key != b_key:
      # print(f'ERROR: {a_id}, {b_id}, {sim}')
      fp += 1
      continue
    if a_key not in res_by_ids:
      res_by_ids[a_key] = set()
    res_by_ids[a_key].add(a_id)
    res_by_ids[a_key].add(b_id)
  for key in expected_res:
    if key not in res_by_ids:
      if len(expected_res[key]) > 1:
        # print(f'ERROR: not found {key}')
        fn += len(expected_res[key])
      continue
    # if len(expected_res[key]) != len(res_by_ids[key]):
    #   print(f'ERROR: not full {key}')
    fn += len(expected_res[key]) - len(res_by_ids[key])
    tp += len(res_by_ids[key])
  precision = 0
  if tp + fp != 0:
    precision = tp / (tp + fp)
  recall = 0
  if tp + fn != 0:
    recall = tp / (tp + fn)
  f1 = 0
  if precision + recall != 0:
    f1 = 2 * precision * recall / (precision + recall)
  return tp, fp, fn, precision, recall, f1

def res_stats(res: List[Tuple[Dict[Hashable, Any], Dict[Hashable, Any]]], expected_res) -> None:
  tp, fp, fn, precision, recall, f1 = get_stats(res, expected_res)
  print(f'tp: {tp}, fp: {fp}, fn: {fn}')
  print(f'precision: {precision}')
  print(f'recall: {recall}')
  print(f'f1: {f1}')

In [17]:
res = dcs_plus_plus(data_list, dcs_key, is_duplicate=is_duplicate, w=20)

In [18]:
res_stats(res, expected_res)

tp: 476, fp: 0, fn: 64
precision: 1.0
recall: 0.8814814814814815
f1: 0.937007874015748


In [19]:
def get_lev_dist(a: Dict[Hashable, Any], b: Dict[Hashable, Any]) -> int:
  a_s = ''
  b_s = ''
  for k in column_names:
    if a[k] is not None and isinstance(a[k], str) and a[k] != '' and b[k] is not None and isinstance(b[k], str) and b[k] != '':
      a_s += a[k]
      b_s += b[k]
  return levenshtein_distance_memopt(a_s, b_s)

In [20]:
from bk_tree import BKTree

In [21]:
# bkt = BKTree(get_lev_dist)
# for r in data_list:
#   bkt.insert(r)

# res: List[Tuple[Dict[Hashable, Any], Dict[Hashable, Any]]] = []
# qres = []
# for org in origs.values():
#   q_res = bkt.query(org, 15)
#   for (_,r) in q_res:
#     if r['rec_id'] != org['rec_id']:
#       res.append((org, r))

# res_stats(res, expected_res)

In [22]:
from bk2 import BKTree as BK2

In [23]:
# bk2 = BK2(get_lev_dist)
# for r in data_list:
#   bk2.insert(r)

# res: List[Tuple[Dict[Hashable, Any], Dict[Hashable, Any]]] = []
# for org in origs.values():
#   q_res = bk2.search(org, 15)
#   for (_,r) in q_res:
#     if r['rec_id'] != org['rec_id']:
#       res.append((org, r))

# res_stats(res, expected_res)

In [24]:
from snm import sorted_neighborhood

In [25]:
def snm_key(r: Dict[Hashable, Any]) -> str:
  name = r['given_name']
  sur = r['surname']
  if name is None:
    name = ''
  elif name != '':
    name = encode(name, max_length=10, zero_pad=True)
  if sur is None:
    sur = ''
  elif sur != '':
    sur.replace(' ', '')
    sur = encode(sur, max_length=10, zero_pad=True)
  s = f'{name}{sur}'
  if s == '':
    return '0'*10
  return s

In [26]:
res = sorted_neighborhood(data_list, snm_key, get_jaccard_sim, 10, 0.7)

In [27]:
res_stats(res, expected_res)

tp: 436, fp: 0, fn: 104
precision: 1.0
recall: 0.8074074074074075
f1: 0.8934426229508198


In [28]:
from ngram import NGramm

In [29]:
def to_str(r: Dict[Hashable, Any]) -> str:
  s = ''
  for k in column_names:
    if r[k] is not None and isinstance(r[k], str) and r[k] != '':
      s += r[k]
  return s

In [30]:
ng = NGramm(data=data_list, to_str=to_str, n=5)

In [31]:
res = ng.find_duplicates(0.45)
res_stats(res, expected_res)

tp: 534, fp: 0, fn: 6
precision: 1.0
recall: 0.9888888888888889
f1: 0.9944134078212291


In [32]:
from winnowing import winnowing_similarity, Winnowing

In [33]:
win = Winnowing(data_list, to_str, 3, 6)

In [34]:
res_stats(win.find_duplicates(0.45), expected_res)

tp: 535, fp: 0, fn: 5
precision: 1.0
recall: 0.9907407407407407
f1: 0.9953488372093023


In [46]:
ks = np.arange(1, 11)
ws = np.arange(1, 11)
nu = np.zeros( (ws.size, ks.size) )
counter_y = 0
for k in ks:
  counter_x = 0
  for w in ws:
    win = Winnowing(data_list, to_str, k, w)
    (_, _, _, _, _, f1) = get_stats(win.find_duplicates(0.65), expected_res)
    nu[counter_x, counter_y] = f1
    counter_x += 1
  counter_y += 1

In [48]:
fig = go.Figure(data=[go.Surface(z=nu, x=ks, y=ws)])
fig.update_layout(title='Mt Bruno Elevation', autosize=False,
                  width=500, height=500)

In [43]:
ts = np.arange(0.1, 0.9, 0.05)
ws = np.arange(1, 11)
nu = np.zeros( (ws.size, ts.size) )
counter_y = 0
for t in ts:
  counter_x = 0
  for w in ws:
    res = sorted_neighborhood(data_list, snm_key, get_jaccard_sim, w, t)
    (_, _, _, _, _, f1) = get_stats(res, expected_res)
    nu[counter_x, counter_y] = f1
    counter_x += 1
  counter_y += 1

In [45]:
fig = go.Figure(data=[go.Surface(z=nu, x=ts, y=ws)])
fig.update_layout(title='Mt Bruno Elevation', autosize=False,
                  width=500, height=500,
                  margin=dict(l=65, r=50, b=65, t=90))